### All needed imports

In [39]:
import os
import cv2
import numpy as np
import matplotlib as pl
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


### Limit GPU Usage

In [40]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


### Preprocessing Data for Model 1

#### Video Paths

In [41]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

#### Processing code, returns frame (num_frames, 224,224,3) and frame_diffs (num_frames, 224,224)

In [42]:
# Code to apply the same random transformation 

In [43]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
            
            
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [44]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    # frame_diffs dimension
    # print(frame_diffs.shape)
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    # frame_diffs_expanded dimension
    # print(frame_diffs_expanded.shape)
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    # combined_input dimension
    # print(combined_input.shape)
    
    # Visualize the difference
    # cv2.imwrite('img_still.png', combined_input[10])
    return combined_input

In [45]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
 # checking data of still_diff
# for i, x in enumerate(still_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)

# checking data of still_diff
# for i, x in enumerate(falling_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)
# print(combined_input)

#data augmentation 
data_augmentation = tf.keras.Sequential([
    # layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(factor=0.2)

])


In [46]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
# checking data of still_diff
# for i, x in enumerate(still_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)

# checking data of still_diff
# for i, x in enumerate(falling_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)

# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
concatenate_labels = concatenate_labels[indices]


# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

#### Create the model

In [47]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')

    # Apply data augmentation here
    x = data_augmentation(combined_input)

    # Convolutional layers for image processing
    # Convolutional layers are useful for learning spatial hierarchies and detecting features 
    x = Conv2D(32, (3, 3), activation='relu')(x) #  Convolutional layers extract spatial features from the image
    x = MaxPooling2D((2, 2))(x) #  Max-pooling layers reduce the size of the feature maps.
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Fully connected layers: These layers learn to make decisions based on the features extracted by the convolutional layers
    x = Flatten()(x) # This line flattens the 2D feature maps into a 1D vector
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss
    
    return model
    
model = create_model()
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 combined_input (InputLayer)  [(None, 224, 224, 4)]    0         
                                                                 
 sequential_3 (Sequential)   (None, 224, 224, 4)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                      

In [48]:
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
54/54 [==============================] - 3s 43ms/step - loss: 13.4079 - accuracy: 0.7847 - val_loss: 0.2437 - val_accuracy: 0.9028
Epoch 2/10
54/54 [==============================] - 2s 41ms/step - loss: 0.2149 - accuracy: 0.9167 - val_loss: 0.1298 - val_accuracy: 0.9583
Epoch 3/10
54/54 [==============================] - 2s 41ms/step - loss: 0.2205 - accuracy: 0.9155 - val_loss: 0.0833 - val_accuracy: 0.9722
Epoch 4/10
54/54 [==============================] - 2s 41ms/step - loss: 0.2102 - accuracy: 0.9248 - val_loss: 0.0750 - val_accuracy: 0.9676
Epoch 5/10
54/54 [==============================] - 2s 41ms/step - loss: 0.1393 - accuracy: 0.9502 - val_loss: 0.0977 - val_accuracy: 0.9537
Epoch 6/10
54/54 [==============================] - 2s 41ms/step - loss: 0.1433 - accuracy: 0.9514 - val_loss: 0.0552 - val_accuracy: 0.9861
Epoch 7/10
54/54 [==============================] - 2s 41ms/step - loss: 0.1840 - accuracy: 0.9444 - val_loss: 0.1621 - val_accuracy: 0.9444
Epoch 8/10
5

In [49]:
# Testing something not part of the main code
arr1 = [[1,2,3], [1,2,3], [1,2,3]]
arr1 = np.array(arr1)
print(arr1.shape)
print(arr1)
print()
# exapand dims through axis 0
arr1_expanded = np.expand_dims(arr1, axis = 0)
print(arr1_expanded.shape)
print(arr1_expanded)
print()
# exapand dims through axis 1
arr2 = [[1,2,3], [1,2,3], [1,2,3]]
arr2 = np.array(arr2)
arr2_expanded = np.expand_dims(arr2, axis = 1)
print(arr2_expanded.shape)
print(arr2_expanded)
print()
arr3 = [[1,2,3], [1,2,3], [1,2,3]]
arr3 = np.array(arr3)
arr3_expanded = np.expand_dims(arr3, axis = -1)
print(arr3_expanded.shape)
print(arr3_expanded)

(3, 3)
[[1 2 3]
 [1 2 3]
 [1 2 3]]

(1, 3, 3)
[[[1 2 3]
  [1 2 3]
  [1 2 3]]]

(3, 1, 3)
[[[1 2 3]]

 [[1 2 3]]

 [[1 2 3]]]

(3, 3, 1)
[[[1]
  [2]
  [3]]

 [[1]
  [2]
  [3]]

 [[1]
  [2]
  [3]]]
